In [20]:
import pandas as pd
import numpy as np
from faker import Faker
import random

# Configuración inicial
fake = Faker()
Faker.seed(42)
random.seed(42)
np.random.seed(42)

# Parámetros de simulación
n_records = 20000  # Número total de registros
n_clients = 1000  # Número de clientes únicos
fraccionadas_ratio = 0.3  # Proporción de transacciones fraccionadas
n_fraccionadas = int(n_records * fraccionadas_ratio)  # Número de transacciones fraccionadas

# Generar clientes únicos
user_ids = [fake.uuid4() for _ in range(n_clients)]

# Crear transacciones fraccionadas
fraccionadas = []
for _ in range(n_fraccionadas // 5):  # Crear grupos de 5 transacciones fraccionadas
    user_id = random.choice(user_ids)
    account_number = fake.bban()
    merchant_id = fake.ean8()
    subsidiary = fake.ean8()
    base_date = fake.date_time_this_year()

    # Generar 5 transacciones en un período de 24 horas
    total_amount = round(np.random.uniform(500, 1000), 2)  # Monto total significativo
    amounts = np.random.dirichlet(np.ones(5), size=1)[0] * total_amount  # Dividir monto
    for amount in amounts:
        fraccionadas.append({
            "_id": fake.uuid4(),
            "merchant_id": merchant_id,
            "subsidiary": subsidiary,
            "transaction_date": base_date + pd.Timedelta(seconds=random.randint(0, 86400)),
            "account_number": account_number,
            "user_id": user_id,
            "transaction_amount": round(amount, 2),
            "transaction_type": "debit",  # Asumimos que las fraccionadas son débitos
        })

# Crear transacciones no fraccionadas
no_fraccionadas = []
n_no_fraccionadas = n_records - len(fraccionadas)
for _ in range(n_no_fraccionadas):
    no_fraccionadas.append({
        "_id": fake.uuid4(),
        "merchant_id": fake.ean8(),
        "subsidiary": fake.ean8(),
        "transaction_date": fake.date_time_this_year(),
        "account_number": fake.bban(),
        "user_id": random.choice(user_ids),
        "transaction_amount": round(np.random.uniform(1, 500), 2),  # Montos menores
        "transaction_type": random.choice(["credit", "debit"]),
    })

# Combinar ambos tipos de transacciones
data_combined = fraccionadas + no_fraccionadas
df_combined = pd.DataFrame(data_combined)

# Ordenar por fecha de transacción
df_combined = df_combined.sort_values(by="transaction_date").reset_index(drop=True)

# Guardar en un archivo CSV
file_path = "synthetic_transactions_final.parquet"
df_combined.to_parquet(f'/mnt/c/Users/Jorge Robledo/Documents/Python/Scripts_and_Notebooks/Proyectos Profesionales/Pruebas técnicas/Nequi/data/{file_path}', index=False)

print(f"Datos generados correctamente y guardados en {file_path}.")

Datos generados correctamente y guardados en synthetic_transactions_final.parquet.


In [116]:
# Display
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

data = pd.read_parquet(f'/mnt/c/Users/Jorge Robledo/Documents/Python/Scripts_and_Notebooks/Proyectos Profesionales/Pruebas técnicas/Nequi/data/{file_path}')

# Verificar si cada account_number está asociado a un único user_id
unique_accounts = data.groupby('user_id')['account_number'].nunique()

# Analizar resultados
if unique_accounts.max() == 1:
    print('La relación es 1:1 (una cuenta por usuario).')
else:
    print('La relación no es 1:1. Hay cuentas compartidas por varios usuarios.')

# Ordenar por account_number, user_id y transaction_date para un análisis secuencial
data = data.sort_values(by=['account_number', 'user_id', 'transaction_date'])

# Crear una columna indicando la diferencia de tiempo entre transacciones consecutivas
data['windows_time'] = data.groupby(['account_number', 'user_id'])['transaction_date'].diff().dt.total_seconds().fillna(0) / 3600

# Agrupar por account_number y user_id para consolidar la información
data = data.groupby(['account_number', 'user_id']).agg(
    windows_time=('windows_time', 'sum'),
    transaction_count=('transaction_date', 'count'),
    transaction_date=('transaction_date', 'mean'),
    transaction_amount=('transaction_amount', 'sum'),
    transaction_type=('transaction_type', lambda x: x.mode()[0] if not x.mode().empty else None),  # Usamos pandas.Series.mode
).reset_index()

# Crear la columna fraction_flag basada en las reglas
data['fraction_flag'] = data.apply(
    lambda row: 'fraccionada' if row['windows_time'] <= 24 and row['transaction_count'] > 1 else 'no fraccionada',
    axis=1
)

data.drop(columns=['windows_time', 'transaction_count'], inplace=True)
data

La relación no es 1:1. Hay cuentas compartidas por varios usuarios.


,account_number,user_id,transaction_date,transaction_amount,transaction_type,fraction_flag
0,AABF72379081113950,1ca3c448-0279-46a6-8f97-97b06d7ce3c9,2025-01-02 09:51:51.628826880,90.50,debit,no fraccionada
1,AABL46199073805045,901bcdef-b56f-48ce-bd66-971e88476c56,2025-01-07 10:41:55.302610944,258.64,debit,no fraccionada
2,AAEG88841649433869,9f708368-cb3c-48ca-a392-57316b79afcc,2025-01-04 00:10:32.930148096,196.79,debit,no fraccionada
3,AAFH61149854104782,4c6a70f4-8ede-444d-876c-f68c3f1be0d0,2025-01-01 07:41:08.506103040,95.11,debit,no fraccionada
4,AAFJ92785234615121,8d244e3e-c4da-4ddb-a0b1-5abaa6a27967,2025-01-08 14:42:41.516628992,921.28,debit,fraccionada
...,...,...,...,...,...,...
15195,ZZVU82915110913519,f8cda88b-436d-46e2-b83c-fe0be037e5ed,2025-01-07 14:31:48.443721984,362.73,credit,no fraccionada
15196,ZZVX10748935558717,23e2fcb4-72d8-467d-894a-05e430b187ef,2025-01-02 08:19:45.090318080,314.03,debit,no fraccionada
15197,ZZWE93426747284460,497ec6d1-081f-46dc-b8d9-a88aef0bea4f,2025-01-01 01:28:22.675819008,307.59,debit,no fraccionada
15198,ZZWK36251231154111,2d174fc9-6f7c-45ea-a72a-6d8eb5122df8,2025-01-04 07:54:49.409979904,724.54,debit,fraccionada


In [117]:
file_path = 'data_final.parquet'
data.to_parquet(f'/mnt/c/Users/Jorge Robledo/Documents/Python/Scripts_and_Notebooks/Proyectos Profesionales/Pruebas técnicas/Nequi/data/{file_path}', index=False)

In [109]:
data['fraction_flag'].value_counts()

fraction_flag
no fraccionada    14000
fraccionada        1200
Name: count, dtype: int64